# Development steps

1. IAM role image handling;
2. ECR image
3. Lambda function creation;
4. API Gateway


## IAM Role Image Handling

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 

## ECR

1. Log in on ECR account;
2. Delete existent ECR image;
3. Create ECR image;
4. Build docker image;
5. Tag docker image;
6. Push docker image to ECR.

## Lambda function

1. Create lambda function;
2. Test lambda function.

## API Gateway

1. Create REST API
2. Create API resource
3. Create REST method
4. Set up integration with the Lambda function
5. Deploy API
6. Create API key
7. Create usage plan
8. Associate the usage plan with the API key
9. Grant API Gateway permission to invoke the Lambda function


# Deployment pipeline

In [1]:
from os import getcwd
from deploy.core import do_deploy

method_verb='POST'

activity_info_ = {
    'image_name': "serverless-example",
    "lambda_function_name": 'test_function',
    'lambda_function_description': 'A test Lambda function',
    "rest_api_name": 'my-api',
    'endpoint': "predict",
    'method_verb': method_verb,
    'stage': "test"
}

# Load environment variables from .env
base_path=getcwd()+'/deploy'

configuration_info_={
    'environment_path': base_path+'/.env',
    'trust_policy_path': base_path,
    'usage_constraints_path': base_path
}

deployment_info=do_deploy(activity_info_, configuration_info_)

Loaded JSON: /home/brunolnetto/github/trouchet/lambda-api/deploy/api_usage_constraints.json
Loaded JSON: /home/brunolnetto/github/trouchet/lambda-api/deploy/trust_policy.json
Starting execution of Deployment of ML solution...
Starting execution of Docker image upload on AWS ECR...
Logging in on ECR account...
Building docker image...
Tagging docker image...
Pushing docker image to ECR...
Finished execution of Docker image upload on AWS ECR.
Time taken: 25.75 seconds
Starting execution of Role policy attachment...
Finished execution of Role policy attachment.
Time taken: 1.00 seconds
Starting execution of API endpoint deployment...
An error occurred (ConflictException) when calling the CreateUsagePlanKey operation: Usage Plan d4vdxp cannot be added because API Key xxnm3md6p0 cannot reference multiple Usage Plans with the same API Stage: 76wtvuoioe:test
Finished execution of API endpoint deployment.
Time taken: 3.10 seconds
Finished execution of Deployment of ML solution.
Time taken: 31.

In [4]:
from json import dump

# Retrieve information from deployment response
api_key=deployment_info['api_key']
api_url = deployment_info['api_url']

deployment_response = {
    'api_key': api_key,
    'api_url': api_url,
    'method_verb': method_verb,
    # Add other information as needed
}

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'w') as json_file:
    dump(deployment_response, json_file)

print(f"Deployment information saved to {deployment_info_filename}")

Deployment information saved to deployment_info.json


In [5]:
from json import load

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'r') as json_file:
    json_payload=load(json_file)

method_verb=json_payload['method_verb']
api_key=json_payload['api_key']
api_url=json_payload['api_url']

print(f"Deployment information loaded from {deployment_info_filename}")

Deployment information loaded from deployment_info.json


In [13]:
from requests import post

def query_post_endpoint(api_url_, api_key_, example_):
    headers = {
        'Content-type': 'application/json', 
        'x-api-key': api_key_,
    }

    resp = post(api_url_, headers=headers, json=example_)

    # Check the response and handle it accordingly
    if resp.status_code == 200:
        response_data = resp.json()
        print(response_data)
    else:
        print("Request failed with status code:", resp.status_code)
        print(resp.text)

In [14]:
import json
import numpy as np

# Prepare the event to pass to the Lambda function
example=[1,2,3,4,5,6,7,8,9]

query_post_endpoint(api_url, api_key, example)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


In [15]:
# WIP

import json
import requests
from about_time import about_time

# Prepare the event to pass to the Lambda function
example_sizes=[1, 10, 100, 1000, 10000, 100000, 100000]
durations=[]

with about_time() as total_t:
    for example_size in example_sizes:
        with about_time() as single_t:
            # Transform into json format
            example=list(range(example_size))
            
            query_post_endpoint(api_url, api_key, example)
        
        durations.append(single_t.duration_human)


Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
